# Image Support

## Course Points

- Claude can analyze images: describe content, compare multiple images, count objects, perform complex visual analysis
- Image limits: max 100 images per request, 5MB per image, 8000px max dimension (single image) or 2000px (multiple images)
- Token calculation: (width px × height px) / 750
- Images included as base64 encoding or URL alongside text blocks in user messages
- **Critical**: Simple prompts produce poor results - apply same prompt engineering techniques as text
- Best practices: detailed guidelines, step-by-step analysis, one-shot examples, break down complex tasks
- Real-world use case: automated fire risk assessment from satellite imagery using structured multi-step prompts

## Code Highlights
```python
# Load and encode image
with open("image.png", "rb") as f:
    image_bytes = base64.standard_b64encode(f.read()).decode("utf-8")

# Add image message with both image and text blocks
add_user_message(messages, [
    {
        "type": "image",
        "source": {
            "type": "base64",
            "media_type": "image/png",
            "data": image_bytes,
        }
    },
    {
        "type": "text",
        "text": "What do you see in this image?"
    }
])
```

In [5]:
# Load env variables and create client
import base64
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-sonnet-4-5"

In [2]:
# Helper functions
from anthropic.types import Message


def add_user_message(messages, message):
    user_message = {
        "role": "user",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(user_message)


def add_assistant_message(messages, message):
    assistant_message = {
        "role": "assistant",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(assistant_message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    thinking=False,
    thinking_budget=1024,
):
    params = {
        "model": model,
        "max_tokens": 4000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if thinking:
        params["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget,
        }

    if tools:
        params["tools"] = tools

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message


def text_from_message(message):
    return "\n".join([block.text for block in message.content if block.type == "text"])

In [3]:
# Fire risk assessment prompt
prompt = """
Analyze the attached satellite image of a property with these specific steps:

1. Residence identification: Locate the primary residence on the property by looking for:
   - The largest roofed structure 
   - Typical residential features (driveway connection, regular geometry)
   - Distinction from other structures (garages, sheds, pools)
   Describe the residence's location relative to property boundaries and other features.

2. Tree overhang analysis: Examine all trees near the primary residence:
   - Identify any trees whose canopy extends directly over any portion of the roof
   - Estimate the percentage of roof covered by overhanging branches (0-25%, 25-50%, 50-75%, 75-100%)
   - Note particularly dense areas of overhang

3. Fire risk assessment: For any overhanging trees, evaluate:
   - Potential wildfire vulnerability (ember catch points, continuous fuel paths to structure)
   - Proximity to chimneys, vents, or other roof openings if visible
   - Areas where branches create a "bridge" between wildland vegetation and the structure
   
4. Defensible space identification: Assess the property's overall vegetative structure:
   - Identify if trees connect to form a continuous canopy over or near the home
   - Note any obvious fuel ladders (vegetation that can carry fire from ground to tree to roof)

5. Fire risk rating: Based on your analysis, assign a Fire Risk Rating from 1-4:
   - Rating 1 (Low Risk): No tree branches overhanging the roof, good defensible space around the structure
   - Rating 2 (Moderate Risk): Minimal overhang (<25% of roof), some separation between tree canopies
   - Rating 3 (High Risk): Significant overhang (25-50% of roof), connected tree canopies, multiple points of vulnerability
   - Rating 4 (Severe Risk): Extensive overhang (>50% of roof), dense vegetation against structure, numerous ember catch points, limited defensible space

For each item above (1-5), write one sentence summarizing your findings, with your final response being the numeric Fire Risk Rating (1-4) with a brief justification.
"""

In [4]:
# TODO: Read image data, feed into Claude
with open("002_images/prop7.png", "rb") as f:
    image_bytes = base64.standard_b64encode(f.read()).decode('utf-8')

messages = []

add_user_message(messages, [
    {
        "type": "image",
        "source": {
            "type": "base64",
            "media_type": "image/png",
            "data": image_bytes
        }
    },
    {
        "type": "text",
        "text": prompt
    }
] )

chat(messages)

Message(id='msg_01XMGhZaULRgc6icpV6Sr3TS', content=[TextBlock(citations=None, text='# Satellite Image Fire Risk Analysis\n\n**1. Residence identification:** The primary residence is a large rectangular structure with a light-colored roof located in the center of the image, surrounded by dense forest vegetation on all sides with no visible driveway or clear property boundaries in this aerial view.\n\n**2. Tree overhang analysis:** Multiple tree canopies directly overhang significant portions of the residence roof, with approximately 50-75% of the roof area covered or shadowed by overhanging branches, particularly dense on the eastern and northern sections of the structure.\n\n**3. Fire risk assessment:** The overhanging trees create multiple ember catch points across the roof surface and establish direct fuel pathways from the surrounding forest canopy to the structure, with no visible defensible space or vegetation clearance around the home.\n\n**4. Defensible space identification:** T